# Introduction
This Notebook will be used for the Applied Data Science Capstone.


In [1]:
#Imports different packages we will need.
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import numpy as np
import requests
import folium
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors


print("Hello Capstone Project Course!")

ModuleNotFoundError: No module named 'folium'

# Part 1

In [ ]:
#Loads data from wiki and puts it into a table using pandas
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
df = pd.read_html(source.content, header=0)[0]

#Changes the Column Headers
header = ['PostalCode', 'Borough', 'Neighborhood']
df.columns = header

#Drops cells with boroughs that are not assigned.
df1 = df[df.Borough != 'Not assigned']

#Replaces Neighborhood name with Borough if it is not assigned
df1.loc[df1.Neighborhood == 'Not assigned']
df1.Neighborhood.replace('Not assigned',df1.Borough,inplace=True)

#Groups Neighborhood and PostalCode
df1 = df1.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(lambda x: ','.join(x))
#Reset Index
df1 = df1.reset_index()

df1.head()

# Part 2

In [ ]:
#Creates a new data frame from a csv file on Geospatial data
web = 'http://cocl.us/Geospatial_data'
df_loc = pd.read_csv(web)

#Joins together the previous data frame and the new data frame on locations
df1 = df1.join(df_loc.set_index('Postal Code'), on='PostalCode')

df1.head()

Part 3: Explore and Cluster the Neighborhoods in Toronto

In [ ]:
#Gets the Coordinates of Toronto
address = 'Toronto'
geolocator = Nominatim(user_agent="Tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

In [ ]:
#Creates a map of Toronto
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

#add markers to the map
for lat, lng, borough, neighborhood in zip(df1['Latitude'], df1['Longitude'], df1['Borough'], df1['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

We will look at the data of boroughs with the word "Toronto" in it

In [ ]:
#Creates a data frame with only data on Scarborough
borough_toronto_data = df1[df1['Borough'].str.contains("Toronto")].reset_index(drop=True)
borough_toronto_data.head()


We will create a map of with only boroughs that contain the word "Toronto"

In [ ]:
#Creates a map of all the boroughs that have "Toronto" in the name using latitude and longitude values
map_borough_toronto = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, borough, neighborhood in zip(
        borough_toronto_data['Latitude'], 
        borough_toronto_data['Longitude'], 
        borough_toronto_data['Borough'], 
        borough_toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_borough_toronto)  

map_borough_toronto


#### Define Foursquare Credentials and Version

In [ ]:
CLIENT_ID = 'H02GZU3YVDIY5GWT3JMWZ0VZQS1K10ZE5DHV13F03PHY5TUB' # your Foursquare ID
CLIENT_SECRET = 'Q1CEIAOSDCGVEFYCZ3FY3303UJVYN04ZFJAAMXQGDT3EAMOR' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

#### Let's explore the first neighborhood in our dataframe.

Get the neighborhood's name.

In [ ]:
borough_toronto_data.loc[0, 'Neighborhood']

Get the neighborhood's latitude and longitude values.

In [ ]:
neighborhood_latitude = borough_toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = borough_toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = borough_toronto_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

#### Now, let's get the top 100 venues that are in Marble Hill within a radius of 500 meters.


First, let's create the GET request URL. Name your URL **url**.

In [ ]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)


Send the GET request and examine the resutls

In [ ]:
results = requests.get(url).json()
results

In [ ]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [ ]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

And how many venues were returned by Foursquare?

In [ ]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

<a id='item2'></a>

# Part 2

#### Let's create a function to repeat the same process to some of the neighborhoods in Toronto

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)



#### Now write the code to run the above function on each neighborhood and create a new dataframe called *manhattan_venues*.

In [ ]:
borough_toronto_venues = getNearbyVenues(names=borough_toronto_data['Neighborhood'],
                                   latitudes=borough_toronto_data['Latitude'],
                                   longitudes=borough_toronto_data['Longitude']
                                  )

print(borough_toronto_venues.shape)
borough_toronto_venues.head()


Let's check how many venues were returned for each neighborhood

In [ ]:
borough_toronto_venues.groupby('Neighborhood').count()

#### Let's find out how many unique categories can be curated from all the returned venues

In [ ]:
print('There are {} uniques categories.'.format(len(borough_toronto_venues['Venue Category'].unique())))

# Part 3

In [ ]:
# one hot encoding
toronto_onehot = pd.get_dummies(borough_toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = borough_toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

In [ ]:
# Gives the size of the new dataframe
toronto_onehot.shape


#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [ ]:
toronto_onehot_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_onehot_grouped

#Confirm new size
toronto_onehot_grouped.shape

#### Let's print each neighborhood along with the top 5 most common venues

In [ ]:
num_top_venues = 5

for hood in toronto_onehot_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_onehot_grouped[toronto_onehot_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')


#### Let's put that into a *pandas* dataframe

In [3]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#Display the top 10 venues for each neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_onehot_grouped['Neighborhood']

for ind in np.arange(toronto_onehot_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_onehot_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()


NameError: name 'toronto_onehot_grouped' is not defined